<a href="https://colab.research.google.com/github/kavyajeetbora/geemap/blob/master/end_to_end_earth_engine/Module_01_Earth_Engine_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

import zipfile
import geopandas as gpd

In [2]:
!pip install PyCRS

  Preparing metadata (setup.py) ... done
  Created wheel for PyCRS: filename=PyCRS-1.0.2-py3-none-any.whl size=32687 sha256=56e1b0a5ef55253ee6d5ae4652a7061c48d50c53255ce10849a9020a13eb9b06
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built PyCRS


## Working with Image Collections

An ImageCollection is a dataset that consists of images takes at different time and locations - usually from the same satellite or data provider.

You can load a collection by searching the Earth Engine Data Catalog for the `ImageCollection ID`. Search for the`Sentinel-2 Level 1C` dataset and you will find its id `COPERNICUS/S2_SR`

**About Sentinel-2 Satellite**

![](https://content.satimagingcorp.com/media2/filer_public_thumbnails/filer_public/b8/3b/b83b4782-bd4f-404f-badf-0bc160cdd959/cms_page_media1530sentinel-2.jpeg__400.0x305.0_q85_subsampling-2.jpg)

The Copernicus Program is an ambitious initiative headed by the European Commission in partnership with the European Space Agency (ESA). The Sentinels are a constellation of satellites developed by ESA to operationalize the Copernicus program, which include all-weather radar images from Sentinel-1A and 1B, high-resolution optical images from Sentinel-2A and 2B, ocean and land data suitable for environmental and climate monitoring from Sentinel-3, as well as air quality data from Sentinel-5P.

Sentinel-2 is a wide-swath, high-resolution, multi-spectral imaging mission supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas.

The Sentinel-2 data contain 13 UINT16 spectral bands representing TOA reflectance scaled by 10000

**What is cloud masking ?**

Cloud masking on GEE using Landsat 8 involves the identification and removal of cloud-contaminated pixels from satellite imagery.

'QA60' band refers to the Quality Assessment band of Sentinel-2 images, which contains information about various atmospheric conditions, including cloud and cirrus cover. This band is used to identify and mask out pixels affected by clouds and cirrus in the image



In [3]:
def maskS2clouds(image):
    '''
    Function to mask clouds using the Sentinel-2 QA band
    @param {ee.Image} image Sentinel-2 image
    @return {ee.Image} cloud masked Sentinel-2 image
    '''
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

dataset = ee.ImageCollection('COPERNICUS/S2').filterDate('2018-01-01', '2018-06-30') \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds)

Map = geemap.Map(center=(12.9407, 77.5925), zoom=12)

viz_params = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2']
}

## Why 3 bands in visualization parameters ? -> computers can visualize only 3 channels i.e. RGB

Map.addLayer(dataset.median(), viz_params, 'RGB')

Map

Map(center=[12.9407, 77.5925], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

How to know the visualization parameters ?

Like what are min, max values ?

Mostly the satellite reflectance values are stored in signed integer format like int8 or int16.

For example int16 ranges from 0 to $2^{16}$ i.e; 65,535

If we store the reflectance in actual values that is float it will take up 4 times more than a integer. Hence to save disk space and data processing time, the reflectance values are stored in integer format.

To convert the values, multiply the values by the scale of each band. The scale for each band for sentinel for example can be found on google earth engine. [link](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2#bands)





## Exercise

Find the 'Sentinel-2 Level-1C' dataset page
https://developers.google.com/earth-engine/datasets

Copy/paste the code snippet

Change the code to display images for your home city

In [4]:
Map = geemap.Map(center=(26.6401,92.7964), zoom=13)

Map.addLayer(dataset, viz_params, name='SENTINEL')
Map

Map(center=[26.6401, 92.7964], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

## Filtering Image Collection

- The collection contains all imagery ever collected by the sensor. The entire collections are not very useful.
- Most applications require a subset of the images. We use filters to select the appropriate images.
-There are many types of filter functions, look at ee.Filter... module to see all available filters. Select a filter and then run the filter() function with the filter parameters

We will learn about 3 main types of filtering techniques

1. **Filter by metadata**: You can apply a filter on the image metadata using filters such as ee.Filter.eq(), ee.Filter.lt() etc. You can filter by PATH/ROW values, Orbit number, Cloud cover etc.
2. **Filter by date**: You can select images in a particular date range using filters such as ee.Filter.date().
3. **Filter by location**: You can select the subset of images with a bounding box, location or geometry using the ee.Filter.bounds(). You can also use the drawing tools to draw a geometry for filtering.


Reference:

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/8sPY5PW31L0/0.jpg)](https://www.youtube.com/watch?v=8sPY5PW31L0)

In [5]:
ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

In [6]:
center = [77.60412933051538, 12.952912912328241]
Map = geemap.Map(center=center, zoom=12)
geometry = ee.Geometry.Point(center)
Map.centerObject(geometry, zoom=10)

s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

## Filter by metadata
filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

## Filter by date
filtered = s2.filter(ee.Filter.date('2019-01-01', '2020-01-01'))

## Filter by location
filtered = s2.filter(ee.Filter.bounds(geometry))

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
  .filter(ee.Filter.bounds(geometry))


viz_params = {
    'bands': ['B4', 'B3', 'B2']
}

Map.addLayer(filtered, viz_params, name="SENTINEL")

# print(filtered.size())

Map

Map(center=[12.952912912328241, 77.60412933051538], controls=(WidgetControl(options=['position', 'transparent_…

Here you can see the image is not clearly visible because we have not set the visulaziation parameters to RGB range. Lets change it so it is visible

In [7]:
viz_params = {
    'min': 0,
    'max':3000,
    'bands': ['B4', 'B3', 'B2']
}

Map.addLayer(filtered, viz_params, name="SENTINEL")

# print(filtered.size())

Map

Map(bottom=121859.0, center=[12.952912912328241, 77.60412933051538], controls=(WidgetControl(options=['positio…

**Satellite Reflectance**

<img src='https://earthstartsbeating.files.wordpress.com/2017/04/olci_reflectance1.gif?resize=720%2C722' height=400/>

Normally the reflectance of objects in urban areas, land is generally in range of 0 to 0.3

Most of the reflectance values are stored in int16 format to save diskspace (pixel value). Inorder to get the actual reflectance values we mulitply it by the scale of different bands
For example, the scale in sentinel satellites is 0.0001 in most of the bands


**Excercise**

In [8]:
center = (92.7964, 26.6401)
point = ee.Geometry.Point(center)

s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
.filter(ee.Filter.date('2023-01-01', '2024-01-01'))\
.filter(ee.Filter.bounds(point))

# print(filtered.size())

Map = geemap.Map(center=center, zoom=12)
vis_params = {
    'min': 0,
    'max':3000,
    'bands': ['B4', 'B3', 'B2']
}
Map.addLayer(filtered, vis_params, name='S2_Harmonized')
Map.centerObject(point)
Map

Map(center=[92.7964, 26.6401], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

## Creating Mosaics and Composites from ImageCollections

The default order of the collection is by date. So when you display the collection, it implicitly creates a mosaic with the latest pixels on top. You can call .mosaic() on a ImageCollection to create a mosaic image from the pixels at the top.

In GEE, you can create composite images using the ee. ImageCollection. mosaic() method. This method combines all the images in the collection to create a single image

We can also create a composite image by applying selection criteria to each pixel from all pixels in the stack. Here we use the median() function to create a composite where each pixel value is the median of all pixels from the stack.

References:

- [12:43](https://www.youtube.com/watch?v=GXklbpAn8Dw&t=763s) - Mosaicking Explained
- [16:19](https://www.youtube.com/watch?v=GXklbpAn8Dw&t=979s) - Composite Creation Process



In [21]:
lat,lon = 12.952912912328241, 77.60412933051538

Map = geemap.Map(center=(lat,lon), zoom=11)
geometry = ee.Geometry.Point((lon,lat))
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
.filter(ee.Filter.date('2019-01-01', '2020-01-01'))\
.filter(ee.Filter.bounds(geometry))

mosaic = filtered.mosaic()

median_composite = filtered.median()

Map.addLayer(mosaic, rgbVis, name='Mosaic')
Map.addLayer(filtered, rgbVis, name='Filtered')
Map.addLayer(median_composite, rgbVis, name='Median Composite')

Map

Map(center=[12.952912912328241, 77.60412933051538], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
center = (77.60412933051538, 12.952912912328241)
Map = geemap.Map(center=center, zoom=11)
geometry = ee.Geometry.Point(center)
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
.filter(ee.Filter.date('2020-01-01', '2021-01-01'))\
.filter(ee.Filter.bounds(geometry))

median_composite = filtered.median()

Map.centerObject(geometry)
Map.addLayer(filtered, rgbVis, name='Filtered')
Map.addLayer(median_composite, rgbVis, name='Median Composite')

Map

Map(center=[77.60412933051538, 12.952912912328241], controls=(WidgetControl(options=['position', 'transparent_…

## Working with feature collections

Feature Collections are similar to Image Collections - but they contain Features, not images. They are equivalent to Vector Layers in a GIS. We can load, filter and display Feature Collections using similar techniques that we have learned so far.

Search for GAUL Second Level Administrative Boundaries and load the collection. This is a global collection that contains all Admin2 boundaries. We can apply a filter using the ADM1_NAME property to get all Admin2 boundaries (i.e. Districts) from a state

In [ ]:
admin2 = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

karnataka = admin2.filter(ee.Filter.eq("ADM1_NAME", 'Karnataka'))

visParams = {'color':'red'}

center = (12.952912912328241, 77.60412933051538)
Map = geemap.Map(center=center, zoom=8)
Map.addLayer(karnataka, visParams, name='Admin Boundary-Karnataka')
Map

Map(center=[12.952912912328241, 77.60412933051538], controls=(WidgetControl(options=['position', 'transparent_…

Exercise

In [ ]:
admin2 = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')
sonitpur = admin2.filter(ee.Filter.eq("ADM2_NAME", 'Sonitpur'))
assam = admin2.filter(ee.Filter.eq("ADM1_NAME", 'Assam'))

center = (26.648,93.318)
Map = geemap.Map(center=center, zoom=8)
Map.addLayer(assam, {'color': 'grey'}, 'All Admin2 Polygons');
Map.addLayer(sonitpur, {'color':'red'}, name='Sonitpur')

Map

Map(center=[26.648, 93.318], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Importing Data

We will now import a shapefile of [Urban Centres](https://bit.ly/ghs-ucdb-shapefile) from JRC’s GHS Urban Centre Database (GHS-UCDB).

- First download the file
- Unzip the ghs_urban_centers.zip

Since the shapefile size is large you may get an error:

`The error Request payload size exceeds the limit: 10485760 bytes is caused if HTTP requests are crossing the limit HTTP request size limit: 10 MB during streaming inserts. This is a hard limit and cannot be increased`

So to curb this error, we can load only part of the data

- Use the property 'CTR_MN_NM' containing country names
- Use the property 'P15' containing 2015 Population

In this case we will load only urban centers in India

In [ ]:
!wget https://bit.ly/ghs-ucdb-shapefile --output-document=urban-centers.zip

try:
    with zipfile.ZipFile('urban-centers.zip', 'r') as zip_ref:
        zip_ref.extractall()
except zipfile.BadZipFile:
    print('Not a zip file or a corrupted zip file')

--2024-03-20 07:42:15--  https://bit.ly/ghs-ucdb-shapefile
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1zdHsop4mryx9-owVAwum1QEGBs2I-Xo9 [following]
--2024-03-20 07:42:15--  https://drive.google.com/uc?export=download&id=1zdHsop4mryx9-owVAwum1QEGBs2I-Xo9
Resolving drive.google.com (drive.google.com)... 173.194.197.113, 173.194.197.102, 173.194.197.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.197.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1zdHsop4mryx9-owVAwum1QEGBs2I-Xo9&export=download [following]
--2024-03-20 07:42:15--  https://drive.usercontent.google.com/download?id=1zdHsop4mryx9-owVAwum1QEGBs2I-Xo9&export=download
Resolving drive.usercontent.google.com (drive.usercontent.goo

In [ ]:
# Read the shapefile using geoapandas
gdb = gpd.read_file('ghs_urban_centers.shp', encoding='latin-1')
# print(gdb.crs)  ## make sure the shapefile is in WGS84 CRS

## Now filter the data (only urban centers in India) and convert it to Earth Engine feature collection object
india = gdb[gdb['CTR_MN_ISO']=='IND']
india = geemap.geopandas_to_ee(india)
print(type(india))

Map = geemap.Map(center=(22.55,78.57), zoom=4)
Map.addLayer(india, {'color':'blue'}, "All urban Areas in India")
Map

<class 'ee.featurecollection.FeatureCollection'>


Map(center=[22.55, 78.57], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

**Exercise**: Apply a filter to select only large urban centers in your country and display it on the ma

In [ ]:
# Read the shapefile using geoapandas
gdb = gpd.read_file('ghs_urban_centers.shp', encoding='latin-1')
# print(gdb.crs)  ## make sure the shapefile is in WGS84 CRS

## Now filter the data (only urban centers in India) and convert it to Earth Engine feature collection object
large_pop_ind = gdb[(gdb['CTR_MN_ISO']=='IND') & (gdb['P15']>1000000)]
large_pop_ind = geemap.geopandas_to_ee(large_pop_ind)

Map = geemap.Map(center=(22.55,78.57), zoom=4)
Map.addLayer(large_pop_ind, {'color':'blue'}, "Large urban Areas in India")
Map

Map(center=[22.55, 78.57], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

## Clipping Images

It is often desirable to clip the images to your area of interest. You can use the `clip()` function to mask out an image using a geometry


While in a Desktop software, clipping is desirable to remove unnecessary portion of a large image and save computation time, in Earth Engine clipping can actually increase the computation time. As described in the [Earth Engine Coding Best Practices guide](https://developers.google.com/earth-engine/guides/best_practices?hl=en#if_you_dont_need_to_clip_dont_use_clip), avoid clipping the images or do it at the end of your script otherwise:

Using clip() unnecessarily will increase computation time. Avoid clip() unless it's necessary to your analysis. If you're not sure, don't clip.

Clip is mostly for visualizing it.

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
filtered = india.filter(ee.Filter.eq('UC_NM_MN', 'Bengaluru'))
print(type(filtered))
geometry = filtered.geometry()
print(type(geometry))

rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4','B3', "B2"]
}

filtered = s2.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",30))\
.filter(ee.Filter.date('2019-01-01','2020-01-01'))\
.filter(ee.Filter.bounds(geometry))

image = filtered.median()
clipped = image.clip(geometry)

center = (12.952912912328241, 77.60412933051538)
Map = geemap.Map(center=center, zoom=10)
Map.addLayer(clipped, rgbVis, name='Urban-Bengaluru')
Map

<class 'ee.featurecollection.FeatureCollection'>
<class 'ee.geometry.Geometry'>


Map(center=[12.952912912328241, 77.60412933051538], controls=(WidgetControl(options=['position', 'transparent_…

**Excercise:** Use the Inspector to find the id of your home city or any urban area of your choice and Change the filter to use the id of the selected feature

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
filtered = india.filter(ee.Filter.eq('UC_NM_MN', 'Tezpur'))
print(type(filtered))
geometry = filtered.geometry()
print(type(geometry))

rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4','B3', "B2"]
}

filtered = s2.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",30))\
.filter(ee.Filter.date('2019-01-01','2020-01-01'))\
.filter(ee.Filter.bounds(geometry))

image = filtered.median()
clipped = image.clip(geometry)

center = (26.7, 92.8)
Map = geemap.Map(center=center, zoom=10)
Map.addLayer(clipped, rgbVis, name='Urban-Tezpur')
Map

<class 'ee.featurecollection.FeatureCollection'>
<class 'ee.geometry.Geometry'>


Map(center=[26.7, 92.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

## Exporting the Data

Earth Engine allows for exporting both vector and raster data to be used in an external program. Vector data can be exported as a CSV or a Shapefile, while Rasters can be exported as GeoTIFF files. We will now export the Sentinel-2 Composite as a GeoTIFF file

In [ ]:
geemap.ee_export_image(
    clipped, filename='clipped.tif', scale=90, file_per_band=True
)

Generating URL ...
Please wait ...
Data downloaded to /content


**Note**: The Export file type for vectors must be one of the following: csv, geojson, json, kml, kmz, shp

In [ ]:
geemap.ee_export_vector(large_pop_ind, filename='large_pop_urban_areas.shp')

Generating URL ...
Please wait ...
Data downloaded to /content/large_pop_urban_areas.shp


## Assignment 1

Load the Night Lights Data for May 2015 and May 2020 with stray light correction. Compare the imagery for your region and find the changes in the city due to COVID-19 effect

In [ ]:
def clip_night_data_by_urban_center(night_img_col, urban_center="Tezpur",year=2015):

    '''
    Returns a clipped image of night data raster for a given urban area and year
    '''

    ## Extract the selected urban center area
    filtered = india.filter(ee.Filter.eq('UC_NM_MN', urban_center))
    geometry = filtered.geometry()

    ## Filter the night image collection by date, bounds and cloudy pixel percentage
    filtered = night_img_col.filter(ee.Filter.date(f'{year}-01-01',f'{year+1}-01-01'))\
    .filter(ee.Filter.bounds(geometry))

    image = filtered.first()
    clipped = image.clip(geometry)

    return clipped


## Extract the selected urban center area
urban_area = india.filter(ee.Filter.eq('UC_NM_MN', 'Tezpur'))
geometry = urban_area.geometry()

night = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG")

night_2015 = clip_night_data_by_urban_center(night, urban_center="Tezpur",year=2015)

night_2020 = clip_night_data_by_urban_center(night, urban_center="Tezpur",year=2020)

vis = {
    'min': 0,
    'max': 60,
    'bands': ['avg_rad']
}

left_layer = geemap.ee_tile_layer(night_2015, vis, name="2015")
right_layer = geemap.ee_tile_layer(night_2020, vis, name="2020")

center = (26.7, 92.8)
Map = geemap.Map(center=center, zoom=12)
Map.split_map(left_layer,right_layer)
Map

Map(center=[26.7, 92.8], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

**Note:**
```python
AttributeError: 'ImageCollection' object has no attribute 'clip
```

So keep in mind to convert the image collection to image before clipping. Basically find the composite of all the images


In [30]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
admin = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

sonitpur = admin.filter(ee.Filter.eq('ADM1_NAME', 'Assam'))\
.filter(ee.Filter.eq('ADM2_NAME', 'Sonitpur'))

geometry = sonitpur.geometry()

filtered_img = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',30))\
.filter(ee.Filter.date('2019-01-01','2020-01-01'))\
.filter(ee.Filter.bounds(geometry))

image = filtered_img.median()
clipped_img = image.clip(geometry)

center = (26.7, 92.8)
Map = geemap.Map(center=center, zoom=10)
Map.addLayer(ee_object=sonitpur, vis_params={'color':'red'}, name='Assam')
viz_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', "B3", "B2"]
}
Map.addLayer(clipped_img, viz_params, name='Sentinel')
Map

Map(center=[26.7, 92.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

## References

1. [Brief introduction on remote sensing](https://semiautomaticclassificationmanual-v5.readthedocs.io/en/latest/remote_sensing.html)

2. [Introduction to Google Earth Engine in Javascript: Youtube playlist](https://www.youtube.com/watch?v=K5GTY9sGe5Y&list=PLLW-qoCMKQsx62nKon2-0PMtGzkxlwN7k)